# Misspelling detection and correction

**NOTE**: This notebook depends upon the the Retrotech dataset. If you have any issues, please rerun the [Setting up the Retrotech Dataset](../ch04/1.setting-up-the-retrotech-dataset.ipynb) notebook.

In [1]:
import sys
sys.path.append('..')
import json
from aips import *
import pandas
import nltk
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import numpy as np
import re
nltk.download('stopwords')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AIPS").getOrCreate()
engine = get_engine()

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Listing 6.12

In [11]:
products_collection = engine.get_collection("products")
query = "moden"
response = engine.spell_check(products_collection, query)

{'modes': 421, 'model': 159, 'modern': 139, 'modem': 56, 'mode6': 9}

In [ ]:
#use the real signals
signals_collection = engine.get_collection("signals")
create_view(signals_collection, "signals")

## Listing 6.13

In [ ]:
### Create user-searchs table each raw represent one search query.
query = """SELECT searches.user AS user,
           LOWER(TRIM(searches.target)) As keyword
           FROM signals AS searches WHERE searches.type = 'query'
           GROUP BY keyword, user"""
query_signals = spark.sql(query).collect()

### Step 1: Tokenize queries and count word frequencies. 
Check word frequency distribution quantiles. The quantile will help decide cut off point for potential misspellings and corrections. 

## Listing 6.14

In [ ]:
stop_words = set(stopwords.words("english"))
word_list = defaultdict(int)

for row in query_signals:
    query = row["keyword"]
    tokenizer = RegexpTokenizer(r'\w+') 
    tokens = tokenizer.tokenize(query)
    for token in tokens:
        if token not in stop_words and len(token) > 3 and not token.isdigit():  #drop stopwords and digit only tokens
            # and only consider token length > 3, since hard to judge whether a very short token is misspelled or not
            word_list[token] += 1

## Listing 6.15

In [ ]:
quantiles_to_check = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
quantile_values = np.quantile(np.array(list(word_list.values())), quantiles_to_check)
quantiles = dict(zip(quantiles_to_check, quantile_values))
quantiles

{0.1: 5.0,
 0.2: 6.0,
 0.3: 8.0,
 0.4: 12.0,
 0.5: 16.0,
 0.6: 25.0,
 0.7: 47.0,
 0.8: 142.20000000000027,
 0.9: 333.2000000000007}

### Step 2: compute metadata needed for word matching. 
consider word with low count as misspelling condidates, with high count as correctly spelled candidates. 

## Listing 6.16

In [ ]:
misspell_candidates = []
correction_candidates = []
misspell_counts = []
correction_counts = []
misspell_length = []
correction_length = []
misspell_initial = []
correction_initial = []
for key, value in word_list.items():
    if value <= quantiles[0.2] : #if value == 1:  # this number based on quantile analysis and the data set, more-likely with user-behvaiour data set to be 1
        misspell_candidates.append(key)
        misspell_counts.append(value)
        misspell_length.append(len(key))
        misspell_initial.append(key[0])
    if value >= quantiles[0.8]:
        correction_candidates.append(key)
        correction_counts.append(value)
        correction_length.append(len(key))
        correction_initial.append(key[0])

## Listing 6.17

In [ ]:
misspell_candidates_df = pd.DataFrame({
    "misspell": misspell_candidates, 
    "misspell_counts": misspell_counts, 
    "misspell_length": misspell_length,
    "initial": misspell_initial})
correction_candidates_df = pd.DataFrame({
    "correction": correction_candidates, 
    "correction_counts": correction_counts, 
    "correction_length": correction_length,
    "initial": correction_initial})

In [ ]:
#Show Results:

misspell_candidates_df.orderBy(col("misspell_counts").desc()).head(10)

AttributeError: 'DataFrame' object has no attribute 'orderBy'

### Step 3: Find potential matches 
based on edit distance and whether word initial is the same or not. 

## Lsting 6.18

In [ ]:
def good_match(len1, len2, edit_dist): #allow longer words have more edit distance
    match = 0
    min_length = min(len1, len2)
    if min_length < 8:
        if edit_dist == 1: match = 1
    elif min_length < 11:
        if edit_dist <= 2: match = 1
    else:
        if edit_dist == 3: match = 1
    return match

## Listing 6.19

In [ ]:
matches_candidates = pd.merge(misspell_candidates_df,
                              correction_candidates_df, on="initial")
#join missepll list with correction list based on whether they share the same initials to reduce matching time. 
matches_candidates["edit_dist"] = matches_candidates.apply(
    lambda row: nltk.edit_distance(row.misspell,row.correction), axis=1)
matches_candidates["good_match"] = matches_candidates.apply(
    lambda row: good_match(row.misspell_length, row.correction_length,
                           row.edit_dist),axis=1)

In [ ]:
matches = matches_candidates[matches_candidates["good_match"] == 1].drop(["initial","good_match"],axis=1)

### Step 4: rank potential matched corrections 
based on edit distance and correction word frequency. shorter edit distance and higher word count will be prefered. only the top one correction is selected for final matching. 

In [ ]:
#matches.sort_values(by=['misspell', 'edit_dist', 'correction_counts'], ascending=[True, True, False])
matches_final = matches.groupby("misspell").first().reset_index()

In [ ]:
#Show Results:
cols = ["misspell", "correction", "misspell_counts", "correction_counts", "edit_dist"]
matches_final.sort_values(by=["correction_counts"], ascending=[False])[cols].head(20)

,misspell,correction,misspell_counts,correction_counts,edit_dist
50,iphone3,iphone,6,16854,1
62,latop,laptop,5,14119,1
61,laptopa,laptop,6,14119,1
136,toucpad,touchpad,6,11550,1
137,touxhpad,touchpad,5,11550,1
148,wirless,wireless,6,10060,1
127,tableta,tablet,6,8260,1
8,cage,case,6,7541,1
10,cape,case,5,7541,1
30,gallaxy,galaxy,6,5839,1


## Listing 6.20

In [ ]:
## Alternative - don't tokenize into individual keywords
stop_words = set(stopwords.words("english"))
word_list = defaultdict(int)

for row in query_signals:
    query = row["keyword"]
    if query not in stop_words and len(query) > 3 and not query.isdigit():
        word_list[query] += 1

# TODO: check listing numbers...
#run Listing 16.12-16.15 again...


Up next: Chapter 7 - [Interpreting Query Intent through Semantic Search](../ch07/1.index-datasets.ipynb)